**Modelo mínimo de geração de recuperação aumentada com o modelo Deepseek.**

**Objetivo**: A tarefa à ser realizada é servir como motor de consulta de informações à partir de arquivos pdf e csv carregado via upload.



---



Para começar precisamos carregar as bibliotecas que serão utilizadas no decorrer do projeto. E para carrega-las precisa utilizar um gerenciador de dependências. neste caso usaremos o pip que é o gerenciador padrão do python.

**Transformers**: É um pacote que contém pré-treinado texto, visão computacional, áudio, vídeo e modelos multimodais para treinamento e inferência.
Use transformers para ajustes finos em modelos e em seus dados, construir aplicações que infere. E para casos de uso em IA generativa com multiplas modalidades.

**Transformers**: is a library of pretrained text, computer vision, audio, video,
and multimodal models for inference and training.
Use Transformers to fine-tune models on your data, build inference applications,
and for generative AI use cases across multiple modalities.

In [ ]:
"""
Instala os pacotes necessários para rodar a inteligência artifical.
"""
!pip install transformers accelerate --quiet
!pip install fitz --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.

Aqui é o import do pacote pymupdf. Para ter certeza se o pacote foi baixado ou não utilizamos o comando print que imprimirá a versão do pacote ou, caso não tenha sido baixado, None.

In [ ]:
# Instalar bibliotecas necessárias
!pip install transformers accelerate PyMuPDF pandas --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 35.3 MB/s eta 0:00:00


Pacote files do Google colab serve para Upload de arquivos locais para dentro da pasta indicada no colab.

In [ ]:
# Upload de arquivos locais para dentro da pasta indicada no colab.
from google.colab import files
uploaded = files.upload()

#
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

Saving Atividade try.pdf to Atividade try.pdf


In [ ]:
import os
import pandas as pd
import fitz  # PyMuPDF

In [ ]:

def extrair_texto_pdf(caminho_pdf):
    texto = ""
    with fitz.open(caminho_pdf) as doc:
        for pagina in doc:
            texto += pagina.get_text()
    return texto

def extrair_texto_csv(caminho_csv):
    df = pd.read_csv(caminho_csv)
    return df.to_string(index=False)

O modelo está sendo carregado direto do Hugging faces pelo nome único chamado de model_id dentro da plataforma.

In [ ]:


model_id = "deepseek-ai/deepseek-coder-1.3b-instruct"

# Carregar o tokenizador e o modelo
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.69G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:

# Detectar nome do arquivo enviado
arquivo_nome = next(iter(uploaded))
extensao = os.path.splitext(arquivo_nome)[1].lower()
texto = ""

# Extrair texto
if extensao == ".pdf":
    texto += extrair_texto_pdf(arquivo_nome)
elif extensao == ".csv":
    texto += extrair_texto_csv(arquivo_nome)
else:
    raise ValueError("Formato não suportado. Use PDF ou CSV.")

**Pipeline**: Tem a função de ser interface para outros sistemas. Uma alternativa para outros pacotes construídos como subprocess, commands, os.popen, popen2, os.system, etc.

In [ ]:
gerador = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

Device set to use cpu


In [ ]:
# print("1º revisão dos artigos:",texto)

In [ ]:

while(True):
  pergunta = input("Digite sua pergunta:")
  if pergunta == "exit":
    break

  # Limitar o texto de entrada (para evitar estouro de memória no Colab)
  # pergunta += texto[:1500]

  prompt = f" Elabore um resumo / texto respondendo a pergunta feita aqui: {pergunta}. com os dados retirados dos documentos coletados dentro desta variável: {texto} "

  # print("2º revisão dos artigos:",texto)

  # Gerar resposta
  resposta = gerador(prompt, max_new_tokens=100, do_sample=True)[0]['generated_text']
  print("🔎 Resposta gerada:\n", resposta)


Digite sua pergunta:Do que o artigo está falando?
🔎 Resposta gerada:
  Elabore um resumo / texto respondendo a pergunta feita aqui: Do que o artigo está falando?. com os dados retirados dos documentos coletados dentro desta variável: Atividade try 
 
1)Faça uma aplicação que pegue diferentes caracteres. E caso seja diferente de do alfabeto 
( a até z ) lance a exceção type error. 
 
 
2) Faça uma aplicação que cada exceção lançada seja guardada em um arquivo log.txt 
 
3) Pegue o primeiro exercício e leia as linhas com for. Se vir mais de 3 linhas emita uma 
mensagem: Tente novamente mais tarde. 
 
 
4) Faça uma aplicação que verifique se a string é um palíndromo. 
 
 
5) Faça uma aplicação que recebe uma lista de strings e retorna uma nova lista de strings 
ordenadas alfabeticamente. 
 
 
6) Faça uma aplicação que recebe uma lista de n�meros e retorna
Digite sua pergunta:Do que fala a questão 1) ?
🔎 Resposta gerada:
  Elabore um resumo / texto respondendo a pergunta feita aqui: Do que

Referências


* https://python.langchain.com/docs/integrations/document_loaders/pymupdf/
* https://pypi.org/project/transformers/
* https://pypi.org/project/accelerate/